In [69]:
import pandas as pd
import opendatasets as od
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
#!pip3 install keras-tuner --upgrade
#!pip3 install keras

In [70]:
od.download("https://www.kaggle.com/saurabhbagchi/ship-and-iceberg-images")

Skipping, found downloaded files in "./ship-and-iceberg-images" (use force=True to force download)


In [71]:
import os
for dirname, _, filenames in os.walk('/home/apprenant/PycharmProjects/ClimateProject---E3/ship-and-iceberg-images/input_data.npz'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

 # Load the data

In [72]:
npz = np.load('/home/apprenant/PycharmProjects/ClimateProject---E3/Data analysis/ship-and-iceberg-images/input_data.npz')

x = npz['X_train']
y = npz['Y_train']
del npz

In [73]:
print('shape of picture: {}'.format(x.shape))
print('shape of picture: {}'.format(y.shape))

shape of picture: (4113, 75, 75, 3)
shape of picture: (4113,)


In [74]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_valtest, y_train, y_valtest = train_test_split(x,y, test_size=0.2 random_state=1, stratify=y)

# Second split the 20% into validation and test sets
X_test, X_val, y_test, y_val = train_test_split(X_valtest, y_valtest, test_size=0.8, random_state=1, stratify=y_valtest)

SyntaxError: invalid syntax (<ipython-input-74-051343f07151>, line 2)

In [ ]:
print(np.array(X_train).shape)
print(np.array(X_val).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_val).shape)
print(np.array(y_test).shape)

In [ ]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=5, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(75,75,3)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=5, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    keras.layers.Dropout(0.2),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=5, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.BatchNormalization(),    
    # output layer    
    keras.layers.Dense(10, activation='sigmoid')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
from tensorflow import keras
import keras_tuner

In [75]:
#creating randomsearch object
tuner = keras_tuner.RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 2,
                    overwrite=True)
# search best parameter
tuner.search(X_train,y_train,epochs=3,validation_data=(X_val,y_val))

Trial 2 Complete [00h 00m 54s]
val_accuracy: 0.5516194105148315

Best val_accuracy So Far: 0.73886638879776
Total elapsed time: 00h 03m 36s
INFO:tensorflow:Oracle triggered exit


In [76]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 71, 71, 58)        4408      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 67, 67, 58)        84158     
_________________________________________________________________
dropout (Dropout)            (None, 67, 67, 58)        0         
_________________________________________________________________
flatten (Flatten)            (None, 260362)            0         
_________________________________________________________________
dense (Dense)                (None, 58)                15101054  
_________________________________________________________________
batch_normalization (BatchNo (None, 58)                232       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

In [79]:
history = model.fit(X_train,y_train,
          epochs=20,
          validation_split=0.1,initial_epoch=0)

Epoch 1/20
70/70 [==============================] - 51s 724ms/step - loss: 0.0420 - accuracy: 0.9851 - val_loss: 1.2196 - val_accuracy: 0.8016
Epoch 2/20
70/70 [==============================] - 45s 645ms/step - loss: 0.0499 - accuracy: 0.9842 - val_loss: 1.1219 - val_accuracy: 0.8057
Epoch 3/20
70/70 [==============================] - 46s 664ms/step - loss: 0.0416 - accuracy: 0.9842 - val_loss: 1.8791 - val_accuracy: 0.7571
Epoch 4/20
70/70 [==============================] - 47s 665ms/step - loss: 0.0269 - accuracy: 0.9910 - val_loss: 1.2649 - val_accuracy: 0.8178
Epoch 5/20
70/70 [==============================] - 47s 673ms/step - loss: 0.0433 - accuracy: 0.9869 - val_loss: 1.0199 - val_accuracy: 0.8178
Epoch 6/20
70/70 [==============================] - 46s 662ms/step - loss: 0.0176 - accuracy: 0.9928 - val_loss: 1.0839 - val_accuracy: 0.8300
Epoch 7/20
70/70 [==============================] - 45s 645ms/step - loss: 0.0220 - accuracy: 0.9932 - val_loss: 1.1383 - val_accuracy: 0.8138

In [81]:


#create a dataframe of the model training history
results = pd.DataFrame(history.history)
results.head()



,loss,accuracy,val_loss,val_accuracy
0,0.042002,0.985135,1.219629,0.801619
1,0.049861,0.984234,1.121918,0.805668
2,0.041568,0.984234,1.879126,0.757085
3,0.026945,0.990991,1.264891,0.817814
4,0.043312,0.986937,1.019869,0.817814
